<a href="https://colab.research.google.com/github/Yishun-Wang/Coursera_Capstone/blob/main/cousera_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [5]:
url='https://www.planning.nsw.gov.au/plans-for-your-area/a-metropolis-of-three-cities/greater-sydney-districts'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [7]:
rows = soup.select('tbody tr')
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
data_frame= pandas.DataFrame(index=range(10),columns=range(5))
rows= soup.find("tbody").find_all("tr")
print(len(rows))
row_counter=0
for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    row_counter+=1
    col_counter=0
    for i in range(0, len(col)):
      data_frame.loc[row_counter,col_counter]=col[i].text
      col_counter+=1
data_frame.dropna(inplace=True)
data_frame.replace({'\n': ''}, inplace=True)
data_frame.columns = data_frame.iloc[0]
data_frame.drop(1,axis=0,inplace=True)
data_frame.reset_index()
suburb_list=data_frame.values.tolist()
suburbs=[]
for suburb in suburb_list:
  suburb = list(filter(None,suburb))
  suburbs.extend(suburb)

## output the list of distrcits suburbs 

len(suburbs)



10


33

Clean this data  drop all \n and clean the first row 

In [9]:
suburb_profile=pandas.DataFrame(columns=['Suburbs', 'latitude','longtitude'])
geolocator = Nominatim(user_agent="my_user_agent")
country ="Au"
my_address = '1600 Pennsylvania Avenue NW Washington, DC 20500'

try:
  for city in suburbs:
    loc = geolocator.geocode(city+','+ country,timeout=None)
    cordinates=loc[1]
    suburb_profile=suburb_profile.append({"Suburbs":city,'latitude':cordinates[0], 'longtitude':cordinates[1]},ignore_index=True)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s")
# for city in suburbs:
#   loc = geolocator.geocode(city+','+ country,timeout=None)
#   cordinates=loc[1]
#   suburb_profile=suburb_profile.append({"Suburbs":city,'latitude':cordinates[0], 'longtitude':cordinates[1]},ignore_index=True)
suburb_profile.sort_values('Suburbs',inplace=True)



# amazon_data = amazon_data.append({"Date":date, "Open":Open, "High":high, "Low":low, "Close":close, "Adj Close":adj_close, "Volume":volume}, ignore_index=True)

Error: geocode failed on input %s with message %s


In [10]:
suburb_profile.reset_index(inplace=True)
del suburb_profile['index']
print(suburb_profile)

Empty DataFrame
Columns: [Suburbs, latitude, longtitude]
Index: []


Get the population, area and other profiles of each sububrs 

In [2]:
# df= pandas.DataFrame()
link = "https://en.wikipedia.org/wiki/Local_government_areas_of_New_South_Wales"
table1 = pandas.read_html(link,header=0)[0]
table2 = pandas.read_html(link,header=0)[1]
len(table1)+len(table2)
profile= [table1, table2]
profile_merge=pandas.concat(profile)
profile_merge.drop(0,axis=0, inplace=True)
profile_merge
row_indes=[]
profile_merge.reset_index()
# profile_merge.drop('Index')
# print(profile_merge.columns)
profile_merge.drop(['Date established','Date established.1','Area[2].1','Area[2].2','Population[2].1','Map'],axis=1, inplace=True)
names= list(profile_merge['Local government area'])

# print(len(names))
# print(len(suburbs))
# print(names)
# for n in names:
#   for i in suburbs:
#     if i in n:


for n in names:
  for i in suburbs:
    if i in n:
      profile_merge.iloc[names.index(n),0]=i



## need to check whether there are names that still unchanged manual change

profile_merge = profile_merge.reset_index()
profile_merge.drop(labels=31,axis=0,inplace=True)
profile_merge.drop(labels='index',axis=1,inplace=True)
profile_merge.replace({'Ku-ring-gai Council':'Ku–ring–gai','Sydney, City of':'City of Sydney','Canterbury-Bankstown, City of':'Canterbury–Bankstown'},inplace=True)
profile_merge.columns=['suburbs', 'km^2','Population']
profile_merge[['latitude','longtitude']]=''
print(suburb_profile)
print(profile_merge.shape)


suburbs=list(suburb_profile['Suburbs'])
new_names= list(profile_merge['suburbs'])
print(len(new_names))
print(len(suburbs))
a=list(set(new_names) - set(suburbs))
print(a)


profile_merge.sort_values('suburbs', ascending=True,inplace=True)
print(profile_merge)
suburb_profile.sort_values('Suburbs', ascending=True,inplace=True)
suburbs=list(suburb_profile['Suburbs'])
new_names= list(profile_merge['suburbs'])
profile_merge['latitude']= suburb_profile['latitude']
print(profile_merge)

print(new_names)
print(suburbs)


NameError: ignored